Tabular Playground Competition 

In [ ]:
import pandas as pd
import os
import sklearn
import numpy as np
import xgboost as xgb
from pathlib import Path
from sklearn.metrics import log_loss
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from google.colab import drive
from zipfile import ZipFile


drive.mount('/content/drive')
df = pd.concat(
    [pd.read_csv(ZipFile('/content/drive/MyDrive/Colab Notebooks/submission_files.zip').open(i)) for i in ZipFile('/content/drive/MyDrive/Colab Notebooks/submission_files.zip').namelist()],
    ignore_index=True, axis =1
)
df = df.sort_values(by=1, axis=1)
df = df.iloc[:, :5000]
groundTruth = pd.read_csv('/content/drive/MyDrive/RE_MACHINE_LEARNING_22/train_labels.csv', index_col='id')
Y = groundTruth
X = df.iloc[:20000, :]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state=42)
TestDf = df.iloc[20000:40000, :]



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Modeling

In [ ]:
def evaluateModel(Xtrain, Ytrain, Xtest, Ytest, model):
  model.fit(Xtrain, Ytrain)
  score = model.score(Xtrain, Ytrain) 
  print("Model Training score: %.2f" %score)
  #kfold = KFold(n_splits=5, shuffle=True)
  #kf_cv_scores = cross_val_score(model, Xtrain, Ytrain, cv=kfold )
  #print("K-fold CV average score: %.2f" % kf_cv_scores.mean())
  ypred = model.predict(Xtest)
  ypred =ypred.reshape(2000,1)
  ypred = ypred.astype(np.float64)
  mse = mean_squared_error(Ytest, ypred)
  print("MSE: %.2f" % mse)
  smape = 1/len(Ytest) * np.sum(2*np.abs(ypred-Ytest)/(np.abs(Ytest)+np.abs(ypred))*100)
  print("SMAPE: %.2f" % smape)
  score = log_loss(Ytest, ypred)
  print("Log_Loss: %.2f" % score)

  

XGBoost Model

In [ ]:
xgbr = xgb.XGBRegression(verbosity=0, eta=0.001, min_child_weight=5, colsample_bylevel=0.001,n_estimators=1000, reg_lambda=50, gamma=0.005, num_boosting_rounds=1000) 
evaluateModel(X_train, Y_train, X_test, Y_test, xgbr)
pipe = make_pipeline(xgbr)
pipe.fit(X, Y)
answers = pipe.predict(TestDf)
submisssionFile = pd.DataFrame({'id': TestDf.index, 'pred': answers}).to_csv('TabularPlaygroundSubmission.csv', index=False)


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model Training score: 0.82
MSE: 0.26
SMAPE: 52.60
Log_Loss: 9.08


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyboardInterrupt: ignored

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss, make_scorer

params = { 'max_depth': [3,6,10],
            'nthreads':[10, 100, 500, 1000],
           'learning_rate': [ 0.5, 0.01, 0.05, 0.1],
           #'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.01, 0.1, 0.7, 1],
           #'min_child_weight':[5,7,10],
           'gamma':[0.1, 0.5, 1, 10],
           'objective':['reg:squarederror']

          }
xgbr = xgb.XGBRegressor(seed = 42)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params, 
                   verbose=3, cv=2)
clf.fit(X, Y)
print("Best parameters:", clf.best_params_)
print("Lowest logloss: ", (-clf.best_score_))

Fitting 2 folds for each of 768 candidates, totalling 1536 fits
[CV 1/2] END colsample_bytree=0.01, gamma=0.1, learning_rate=0.5, max_depth=3, nthreads=10, objective=reg:squarederror;, score=0.197 total time=  26.3s
[CV 2/2] END colsample_bytree=0.01, gamma=0.1, learning_rate=0.5, max_depth=3, nthreads=10, objective=reg:squarederror;, score=0.200 total time=  29.0s
[CV 1/2] END colsample_bytree=0.01, gamma=0.1, learning_rate=0.5, max_depth=3, nthreads=100, objective=reg:squarederror;, score=0.197 total time=  25.7s
[CV 2/2] END colsample_bytree=0.01, gamma=0.1, learning_rate=0.5, max_depth=3, nthreads=100, objective=reg:squarederror;, score=0.200 total time=  25.6s
[CV 1/2] END colsample_bytree=0.01, gamma=0.1, learning_rate=0.5, max_depth=3, nthreads=500, objective=reg:squarederror;, score=0.197 total time=  32.3s
[CV 2/2] END colsample_bytree=0.01, gamma=0.1, learning_rate=0.5, max_depth=3, nthreads=500, objective=reg:squarederror;, score=0.200 total time=  26.2s
[CV 1/2] END colsamp

In [ ]:
x = X.copy()
x = pd.get_dummies(x)


X_train, X_test, y_train, y_test = train_test_split(x, Y, test_size = .3, random_state=67)
X_train, X_val, y_train, y_val = train_test_split(x, Y, test_size = .2, random_state=123)
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label = y_val)
dtest = xgb.DMatrix(X_test, label=y_test)
print(X_test.shape)
print(y_test.shape)

xgb_params={"learning_rate": 0.001, 'lambda':0.01}
xmodel = xgb.train(xgb_params, dtrain, num_boost_round=750, evals=[(dval, 'evals')])
y_pred = xmodel.predict(dtest)
print(y_test.shape)
#print(mean_squared_error(y_test, y_pred))
#smape = 1/len(y_test) * np.sum(2*np.abs(y_pred-y_test)/(np.abs(y_test)+np.abs(y_pred))*100)
#print("SMAPE: %.2f" % smape)
#score = log_loss(Y_test, y_pred)
#print("Log_Loss: %.2f" % score)
x_test = pd.get_dummies(TestDf)
dtest = xgb.DMatrix(x_test)
answers = xmodel.predict(dtest)
submisssionFile = pd.DataFrame({'id': TestDf.index, 'pred': answers}).to_csv('XGBTabularPlaygroundSubmission.csv', index=False)

(6000, 5000)
(6000, 1)
[0]	evals-rmse:0.499857
[1]	evals-rmse:0.499714
[2]	evals-rmse:0.499573
[3]	evals-rmse:0.499433
[4]	evals-rmse:0.49929
[5]	evals-rmse:0.499148
[6]	evals-rmse:0.499007
[7]	evals-rmse:0.498865
[8]	evals-rmse:0.498725
[9]	evals-rmse:0.498585
[10]	evals-rmse:0.498443
[11]	evals-rmse:0.498304
[12]	evals-rmse:0.498163
[13]	evals-rmse:0.498023
[14]	evals-rmse:0.497881
[15]	evals-rmse:0.497742
[16]	evals-rmse:0.497602
[17]	evals-rmse:0.497462
[18]	evals-rmse:0.497324
[19]	evals-rmse:0.497183
[20]	evals-rmse:0.497045
[21]	evals-rmse:0.496908
[22]	evals-rmse:0.496771
[23]	evals-rmse:0.496633
[24]	evals-rmse:0.496494
[25]	evals-rmse:0.496353
[26]	evals-rmse:0.496214
[27]	evals-rmse:0.496078
[28]	evals-rmse:0.495937
[29]	evals-rmse:0.495799
[30]	evals-rmse:0.495661
[31]	evals-rmse:0.495525
[32]	evals-rmse:0.495386
[33]	evals-rmse:0.495246
[34]	evals-rmse:0.495113
[35]	evals-rmse:0.494976
[36]	evals-rmse:0.494836
[37]	evals-rmse:0.494704
[38]	evals-rmse:0.494568
[39]	evals-rm

Decision Tree Regression

In [ ]:
DecTreeReg = DecisionTreeRegressor(random_state=42)
evaluateModel(X_train, Y_train, X_test, Y_test, DecTreeReg)
pipe = make_pipeline(DecTreeReg)
pipe.fit(X, Y)
answers = pipe.predict(TestDf)
submisssionFile = pd.DataFrame({'id': TestDf.index, 'pred': answers}).to_csv('DECTREETabularPlaygroundSubmission.csv', index=False)

Random Forest Regression

In [ ]:
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
evaluateModel(X_train,  np.ravel(Y_train), X_test, Y_test, regressor)
regressor.fit(X_train, np.ravel(Y_train))
pipe = make_pipeline(regressor)
pipe.fit(X, Y)
answers = pipe.predict(TestDf)
submisssionFile = pd.DataFrame({'id': TestDf.index, 'pred': answers}).to_csv('RandomForestTabularPlaygroundSubmission.csv', index=False)


Model Training score: 0.86
MSE: 0.20
SMAPE: 112.55
Log_Loss: 0.88


/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
